In [4]:
from spacy.en import English
parser = English()
from nltk.corpus import treebank


In [11]:
sents = []
for f in treebank.fileids():
  count = 0
  for sent in treebank.sents(f):
    count += 1
    sent = ' '.join(sent).strip()
    sents.append(sent)
    

In [60]:
import csv

# load symbols and company names
def load_file1(f_name):
  f = open(f_name)
  f.readline()
  reader = csv.reader(f, delimiter=',', quotechar='"')
  symbols, names = [], []
  for line in reader:
    symbols.append(line[0])
    names.append(line[1])
  return (symbols, names)

# load cusip, isin, figi
def load_file2(f_name):
  f = open(f_name)
  f.readline()
  reader = csv.reader(f, delimiter=',', quotechar='"')
  cusips, isins, figis = [], [], []
  for line in reader:
    cusips.append(line[1])
    isins.append(line[2])
    figis.append(line[3])
  return (cusips, isins, figis)

from enum import Enum
class IDType(Enum):
  Ticker = 1
  Symbol = 2
  CompName = 3
  CUSIP = 4
  ISIN = 5
  FIGI = 6


In [76]:
def get_random_id_generator():
  symbols, names = load_file1('../data/NYSE.csv')
  symbols2, names2 = load_file1('../data/NASDAQ.csv')
  names.extend(names2)
  symbols.extend(symbols2)
  names, symbols = list(set(names)), list(set(symbols))
  
  cusips, isins, figis = load_file2('../data/bond_data.csv')
  cusips, isins, figis = list(set(cusips)), list(set(isins)), list(set(figis))
    
  def fun(id_type):
    assert type(id_type) is IDType, 'Must pass in an IDType'
    if(id_type == IDType.Symbol or id_type == IDType.Ticker):
      return random.sample(symbols, 1)[0]
    elif(id_type == IDType.CompName):
      return random.sample(names, 1)[0]
    elif(id_type == IDType.CUSIP):
      return random.sample(cusips, 1)[0]
    elif(id_type == IDType.ISIN):
      return random.sample(isins, 1)[0]
    elif(id_type == IDType.FIGI):
      return random.sample(figis, 1)[0]
  return fun

get_random_id = get_random_id_generator()

In [78]:
a = [1,2,3,4]
a.extend(a)
a

[1, 2, 3, 4, 1, 2, 3, 4]

In [84]:
def parse(parser, sent):
  doc = parser(sent)
  ents = list(doc.ents)
  new_sent = []
  last_end = 0
  for entity in ents:
    if(entity.label_ == 'ORG'):
      print "ORG"
      new_sent.extend([token.orth_ for token in doc[last_end:entity.start]])
      new_sent.append(get_random_id(IDType.CUSIP))
      last_end = entity.end
  new_sent.extend([token.orth_ for token in doc[last_end:]])
      #print type(entity), entity.start, entity.end
  # print new_sent
  return ' '.join(new_sent).strip()

for sent in sents[1:10]:
  print sent
  print parse(parser, sent)
  print('\n\n')

Mr. Vinken is chairman of Elsevier N.V. , the Dutch publishing group .
ORG
Mr. Vinken is chairman of 228227BE3 , the Dutch publishing group .



Rudolph Agnew , 55 years old and former chairman of Consolidated Gold Fields PLC , was named *-1 a nonexecutive director of this British industrial conglomerate .
ORG
Rudolph Agnew , 55 years old and former chairman of 90272V137 , was named * -1 a nonexecutive director of this British industrial conglomerate .



A form of asbestos once used * * to make Kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed * to it more than 30 years ago , researchers reported 0 *T*-1 .
ORG
A form of asbestos once used * * to make 755111BU4 cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed * to it more than 30 years ago , researchers reported 0 * T*-1 .



The asbestos fiber , crocidolite , is unusually resilient once it enters the lungs , with even brief exposures to